In [2]:
import pandas as pd
import numpy as np

In [14]:
data=pd.read_csv("Combined Data.csv", engine='python', on_bad_lines='warn')

/tmp/ipython-input-1534460413.py:1: ParserWarning: Skipping line 24512: unexpected end of data

  data=pd.read_csv("Combined Data.csv", engine='python', on_bad_lines='warn')


In [15]:
data.shape

(24510, 3)

In [16]:
data.columns

Index(['Unnamed: 0', 'statement', 'status'], dtype='object')

In [22]:
for col in data.columns:
    print(repr(col))


'statement'
'status'


In [23]:
X=data["statement"]
Y=data["status"]

In [25]:
Y.value_counts()

,count
status,
Depression,8957
Suicidal,8571
Normal,6249
Anxiety,733


In [26]:
y_binary = Y.apply(lambda x: 1 if x == "Depression" else 0)
y_binary.value_counts()


,count
status,
0,15553
1,8957


In [27]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    return text

X_clean = X.apply(clean_text)
X_clean.head()


AttributeError: 'float' object has no attribute 'lower'

In [28]:
data = data.dropna(subset=["statement"])


In [29]:
X=data["statement"]
Y=data["status"]

In [30]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    return text


In [31]:
X_clean = X.apply(clean_text)
X_clean.head()


,statement
0,oh my gosh
1,trouble sleeping confused mind restless heart ...
2,all wrong back off dear forward doubt stay in ...
3,ive shifted my focus to something else but im ...
4,im restless and restless its been a month now ...


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words="english"
)

X_tfidf = vectorizer.fit_transform(X_clean)
X_tfidf.shape


(24500, 5000)

In [33]:
y_binary = Y.apply(lambda x: 1 if x == "Depression" else 0)
y_binary.value_counts()


,count
status,
0,15543
1,8957


In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf,
    y_binary,
    test_size=0.2,
    random_state=42,
    stratify=y_binary
)


In [35]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [36]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Accuracy: 0.7802040816326531

Classification Report:

              precision    recall  f1-score   support

           0       0.80      0.87      0.83      3109
           1       0.73      0.63      0.68      1791

    accuracy                           0.78      4900
   macro avg       0.77      0.75      0.76      4900
weighted avg       0.78      0.78      0.78      4900



In [37]:
import joblib

joblib.dump(model, "depression_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']

In [38]:
loaded_model = joblib.load("depression_model.pkl")
loaded_vectorizer = joblib.load("tfidf_vectorizer.pkl")

sample = ["i feel hopeless and exhausted all the time"]
sample_vec = loaded_vectorizer.transform(sample)
loaded_model.predict(sample_vec)


array([0])